In [50]:
import pandas as pd
import numpy as np

from sklearn.utils import resample
from sklearn.model_selection import train_test_split

import xgboost as xgb

from sklearn.metrics import roc_auc_score
# ROC is used for evaluation of model

In [51]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_output = pd.DataFrame()
df_output['id'] = df_test['id'].copy()

In [52]:
df_train.describe()

,id,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,loan_status
count,58645.000000,58645.000000,5.864500e+04,58645.000000,58645.000000,58645.000000,58645.000000,58645.000000,58645.000000
mean,29322.000000,27.550857,6.404617e+04,4.701015,9217.556518,10.677874,0.159238,5.813556,0.142382
std,16929.497605,6.033216,3.793111e+04,3.959784,5563.807384,3.034697,0.091692,4.029196,0.349445
min,0.000000,20.000000,4.200000e+03,0.000000,500.000000,5.420000,0.000000,2.000000,0.000000
25%,14661.000000,23.000000,4.200000e+04,2.000000,5000.000000,7.880000,0.090000,3.000000,0.000000
50%,29322.000000,26.000000,5.800000e+04,4.000000,8000.000000,10.750000,0.140000,4.000000,0.000000
75%,43983.000000,30.000000,7.560000e+04,7.000000,12000.000000,12.990000,0.210000,8.000000,0.000000
max,58644.000000,123.000000,1.900000e+06,123.000000,35000.000000,23.220000,0.830000,30.000000,1.000000


In [53]:
def remove_outliers(df : pd.DataFrame, columns : list, significance: int):
    for column in columns:
        upperLimit = df[column].quantile(1-significance*.01)
        lowerLimit = df[column].quantile(significance*.01)
        df = df.loc[(df[column] > lowerLimit) & (df[column] < upperLimit)]
    return df

def normalize_data(df : pd.DataFrame, columns : list):
    for column in columns:
        df[column] = df[column] / df[column].abs().max() 
    return df

def create_dummies(df : pd.DataFrame, columns : list):
    for column in columns:
        df = pd.concat([df.drop(column, axis=1), pd.get_dummies(df[column], prefix=column)], axis=1)
    return df

In [54]:
columns_outliers = ['person_income', 'loan_amnt', 'person_age', 'person_emp_length', 'loan_int_rate', 'loan_percent_income', 'cb_person_cred_hist_length']
columns_normalization = ['person_income', 'loan_amnt', 'person_age', 'person_emp_length', 'loan_percent_income', 'cb_person_cred_hist_length']
columns_dummies = ['person_home_ownership', 'loan_intent', 'loan_grade']

#df_train = remove_outliers(df_train, columns_outliers, 1)
df_train = normalize_data(df_train, columns_normalization)
df_train = create_dummies(df_train, columns_dummies)
df_train['cb_person_default_on_file'] = df_train['cb_person_default_on_file'].map(dict(Y=1, N=0))

df_test = normalize_data(df_test, columns_normalization)
df_test = create_dummies(df_test, columns_dummies)
df_test['cb_person_default_on_file'] = df_test['cb_person_default_on_file'].map(dict(Y=1, N=0))

In [55]:
df_train['loan_status'].value_counts()

loan_status
0    50295
1     8350
Name: count, dtype: int64

In [56]:
# Balance Dataset by oversampling
minority_class = df_train[df_train['loan_status'] == 1]
majority_class = df_train[df_train['loan_status'] == 0]


minority_upsampled = resample(minority_class, replace=True, n_samples=len(majority_class), random_state=42)
df_train = pd.concat([majority_class, minority_upsampled])

In [57]:
df_train['loan_status'].value_counts()

loan_status
0    50295
1    50295
Name: count, dtype: int64

In [58]:
df_train.head()

,id,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status,...,loan_intent_MEDICAL,loan_intent_PERSONAL,loan_intent_VENTURE,loan_grade_A,loan_grade_B,loan_grade_C,loan_grade_D,loan_grade_E,loan_grade_F,loan_grade_G
0,0,0.300813,0.018421,0.000000,0.171429,11.49,0.204819,0,0.466667,0,...,False,False,False,False,True,False,False,False,False,False
1,1,0.178862,0.029474,0.048780,0.114286,13.35,0.084337,0,0.066667,0,...,True,False,False,False,False,True,False,False,False,False
2,2,0.235772,0.015158,0.065041,0.171429,8.90,0.253012,0,0.333333,0,...,False,True,False,True,False,False,False,False,False,False
3,3,0.243902,0.036842,0.113821,0.342857,11.11,0.204819,0,0.166667,0,...,False,False,True,False,True,False,False,False,False,False
4,4,0.178862,0.031579,0.016260,0.171429,6.92,0.120482,0,0.100000,0,...,True,False,False,True,False,False,False,False,False,False


In [59]:
df_train, df_validation = train_test_split(df_train, test_size=0.2, random_state=42)

In [60]:
clf = xgb.XGBClassifier(tree_method="hist")
clf.fit(df_train.loc[:, df_train.columns != 'loan_status'], df_train['loan_status'])
predictions = clf.predict(df_validation.loc[:, df_validation.columns != 'loan_status'])

In [61]:
roc_auc_score(df_validation['loan_status'],predictions)

0.9379907200177358

In [62]:
predictions_test = clf.predict(df_test)
df_output['loan_status'] = predictions_test
df_output = df_output.set_index('id')
df_output.to_csv('Predictions_v3.csv')
